# Sequence Trading System - Complete Training Pipeline

This notebook demonstrates the end-to-end training pipeline for the Sequence deep learning FX/crypto trading system.

**Features:**
- Asset-class aware preprocessing (FX vs Crypto)
- Multi-GPU training with DataParallel
- Automatic Mixed Precision (AMP)
- Real-time training visualization
- GPU utilization monitoring

**Workflow:**
1. Environment setup and GPU detection
2. Data preprocessing with asset detection
3. Feature engineering and dataset creation
4. Model training with multi-GPU support
5. Training visualization and metrics
6. Model evaluation
7. Crypto-specific analysis

## 1. Setup & Environment Check

In [ ]:
# System setup
import sys
from pathlib import Path

# Add project root to path
ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(ROOT))
sys.path.insert(0, str(ROOT / 'run'))

print(f"Project root: {ROOT}")

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from IPython.display import clear_output

# Project imports
from config.config import (
    AssetConfig, AssetClass, DataConfig, 
    FeatureConfig, ModelConfig, TrainingConfig
)
from utils.multi_gpu import setup_multi_gpu, print_gpu_utilization, get_gpu_memory_info
from train.features.agent_features import build_feature_frame

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

In [ ]:
# Check GPU availability
print("="*60)
print("GPU Environment Check")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    print("\nGPU Details:")
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"  GPU {i}: {props.name}")
        print(f"    Memory: {props.total_memory / 1024**3:.2f} GB")
        print(f"    Compute Capability: {props.major}.{props.minor}")
    
    print("\nCurrent GPU Utilization:")
    print_gpu_utilization()
else:
    print("\n⚠️  No GPU detected. Training will use CPU.")
    print("For best performance, run on a GPU-enabled environment.")

print("="*60)

## 2. Asset-Class Detection Demo

Demonstrate automatic asset class detection and appropriate configuration selection.

In [ ]:
# Test asset detection on various pairs
test_pairs = [
    'gbpusd', 'eurusd', 'usdjpy',  # FX majors
    'btcusd', 'ethusd', 'solusd',  # Crypto
    'xauusd', 'xagusd'             # Commodities
]

print("Asset Class Detection Results")
print("="*80)
print(f"{'Pair':<12} {'Asset Class':<12} {'SMA Windows':<20} {'RSI':<6} {'ATR':<6} {'DC Threshold'}")
print("-"*80)

for pair in test_pairs:
    asset_class = AssetConfig.detect_from_pair(pair)
    asset_cfg = AssetConfig(asset_class=asset_class)
    feature_cfg = asset_cfg.get_feature_config()
    
    print(f"{pair:<12} {asset_class.value:<12} {str(feature_cfg.sma_windows):<20} "
          f"{feature_cfg.rsi_window:<6} {feature_cfg.atr_window:<6} {feature_cfg.dc_threshold_up}")

print("="*80)

## 3. Data Preprocessing Example

Load sample data and apply asset-aware feature engineering.

In [ ]:
# Create sample OHLCV data for demonstration
def create_sample_data(n_samples=1000, pair='gbpusd', seed=42):
    """Create synthetic OHLCV data for testing."""
    np.random.seed(seed)
    
    # Generate timestamps
    dates = pd.date_range('2024-01-01', periods=n_samples, freq='1min')
    
    # Generate price data with realistic characteristics
    if 'btc' in pair.lower() or 'eth' in pair.lower():
        # Crypto: higher volatility
        base_price = 50000 if 'btc' in pair.lower() else 3000
        volatility = 0.002
    else:
        # FX: lower volatility
        base_price = 1.25
        volatility = 0.0001
    
    # Random walk with drift
    returns = np.random.normal(0.00001, volatility, n_samples)
    close = base_price * np.exp(np.cumsum(returns))
    
    # Generate OHLC from close
    high_offset = np.abs(np.random.normal(0, volatility * base_price, n_samples))
    low_offset = np.abs(np.random.normal(0, volatility * base_price, n_samples))
    
    high = close + high_offset
    low = close - low_offset
    open_price = close + np.random.normal(0, volatility * base_price / 2, n_samples)
    
    # Generate volume
    volume = np.random.lognormal(10, 0.5, n_samples)
    
    df = pd.DataFrame({
        'datetime': dates,
        'open': open_price,
        'high': high,
        'low': low,
        'close': close,
        'volume': volume
    })
    
    return df

# Generate sample data for both FX and crypto
fx_data = create_sample_data(n_samples=1000, pair='gbpusd')
crypto_data = create_sample_data(n_samples=1000, pair='btcusd')

print("Sample Data Generated")
print(f"FX data shape: {fx_data.shape}")
print(f"Crypto data shape: {crypto_data.shape}")
print("\nFX Data Preview:")
print(fx_data.head())

In [ ]:
# Apply asset-aware feature engineering
def process_with_asset_detection(df, pair):
    """Apply feature engineering with automatic asset detection."""
    # Detect asset class
    asset_class = AssetConfig.detect_from_pair(pair)
    asset_cfg = AssetConfig(asset_class=asset_class)
    feature_cfg = asset_cfg.get_feature_config()
    
    print(f"Processing {pair}:")
    print(f"  Asset class: {asset_class.value}")
    print(f"  SMA windows: {feature_cfg.sma_windows}")
    print(f"  RSI window: {feature_cfg.rsi_window}")
    
    # Build features
    features_df = build_feature_frame(df.copy(), config=feature_cfg)
    
    print(f"  Features generated: {features_df.shape[1]} columns")
    print(f"  Samples after NaN drop: {len(features_df)}")
    print()
    
    return features_df, feature_cfg

# Process both datasets
fx_features, fx_config = process_with_asset_detection(fx_data, 'gbpusd')
crypto_features, crypto_config = process_with_asset_detection(crypto_data, 'btcusd')

## 4. Feature Analysis & Visualization

In [ ]:
# Compare FX vs Crypto feature distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('FX vs Crypto Feature Comparison', fontsize=16, fontweight='bold')

# Plot 1: Price movement
axes[0, 0].plot(fx_data['close'].values[:200], label='FX (GBPUSD)', alpha=0.7)
axes[0, 0].set_title('FX Price Movement')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Price')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(crypto_data['close'].values[:200], label='Crypto (BTCUSD)', alpha=0.7, color='orange')
axes[0, 1].set_title('Crypto Price Movement')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Price')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 2: RSI comparison
if 'rsi_14' in fx_features.columns:
    axes[1, 0].plot(fx_features['rsi_14'].values[:200], label='FX RSI(14)', alpha=0.7)
    axes[1, 0].axhline(y=70, color='r', linestyle='--', alpha=0.5, label='Overbought')
    axes[1, 0].axhline(y=30, color='g', linestyle='--', alpha=0.5, label='Oversold')
    axes[1, 0].set_title('FX RSI Indicator')
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('RSI')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

if 'rsi_7' in crypto_features.columns:
    axes[1, 1].plot(crypto_features['rsi_7'].values[:200], label='Crypto RSI(7)', alpha=0.7, color='orange')
    axes[1, 1].axhline(y=70, color='r', linestyle='--', alpha=0.5, label='Overbought')
    axes[1, 1].axhline(y=30, color='g', linestyle='--', alpha=0.5, label='Oversold')
    axes[1, 1].set_title('Crypto RSI Indicator (Faster)')
    axes[1, 1].set_xlabel('Time')
    axes[1, 1].set_ylabel('RSI')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print("  • FX uses slower indicators (RSI14, SMA[10,20,50]) for stable markets")
print("  • Crypto uses faster indicators (RSI7, SMA[5,10,20]) for volatile markets")
print("  • Asset detection automatically applies appropriate parameters")

## 5. Multi-GPU Training Setup

Demonstrate multi-GPU model setup and memory management.

In [ ]:
# Create a sample model
class SimpleTradingModel(nn.Module):
    """Simple trading model for demonstration."""
    def __init__(self, num_features=50, hidden_size=128, num_classes=3):
        super().__init__()
        self.lstm = nn.LSTM(num_features, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # x shape: (batch, seq_len, features)
        lstm_out, _ = self.lstm(x)
        # Take last timestep
        last_out = lstm_out[:, -1, :]
        return self.fc(last_out)

# Create model
model = SimpleTradingModel(num_features=50, hidden_size=128, num_classes=3)
print(f"Model created with {sum(p.numel() for p in model.parameters()):,} parameters")

# Setup for multi-GPU
model, device = setup_multi_gpu(model, device="cuda")
print(f"\nModel device: {device}")
print(f"Model type: {type(model).__name__}")

if torch.cuda.is_available():
    print("\nGPU Memory After Model Load:")
    print_gpu_utilization()

## 6. Training Loop with Visualization

Demonstrate training with real-time metrics visualization.

In [ ]:
# Mock training loop with visualization
def train_with_visualization(model, device, epochs=10):
    """Training loop with real-time visualization."""
    train_losses = []
    val_losses = []
    
    # Simulated training
    for epoch in tqdm(range(epochs), desc="Training"):
        # Simulate training metrics
        train_loss = 1.0 * np.exp(-epoch * 0.1) + np.random.normal(0, 0.05)
        val_loss = 1.2 * np.exp(-epoch * 0.08) + np.random.normal(0, 0.08)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Real-time plotting every 2 epochs
        if (epoch + 1) % 2 == 0:
            clear_output(wait=True)
            
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))
            
            # Loss plot
            ax1.plot(train_losses, label='Train Loss', marker='o')
            ax1.plot(val_losses, label='Val Loss', marker='s')
            ax1.set_xlabel('Epoch')
            ax1.set_ylabel('Loss')
            ax1.set_title('Training Progress')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            
            # GPU utilization (if available)
            if torch.cuda.is_available():
                gpu_info = get_gpu_memory_info()
                if gpu_info:
                    gpu_ids = list(gpu_info.keys())
                    allocated = [gpu_info[i]['allocated'] for i in gpu_ids]
                    total = [gpu_info[i]['total'] for i in gpu_ids]
                    
                    x = np.arange(len(gpu_ids))
                    width = 0.35
                    
                    ax2.bar(x - width/2, allocated, width, label='Allocated', alpha=0.8)
                    ax2.bar(x + width/2, total, width, label='Total', alpha=0.5)
                    ax2.set_xlabel('GPU ID')
                    ax2.set_ylabel('Memory (MB)')
                    ax2.set_title('GPU Memory Usage')
                    ax2.set_xticks(x)
                    ax2.set_xticklabels([f'GPU {i}' for i in gpu_ids])
                    ax2.legend()
                    ax2.grid(True, alpha=0.3, axis='y')
            else:
                ax2.text(0.5, 0.5, 'No GPU Available\nTraining on CPU', 
                        ha='center', va='center', fontsize=14)
                ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  Train Loss: {train_loss:.4f}")
            print(f"  Val Loss: {val_loss:.4f}")
    
    return train_losses, val_losses

# Run training demonstration
print("Starting training demonstration...\n")
train_losses, val_losses = train_with_visualization(model, device, epochs=10)

## 7. Summary & Next Steps

### Key Features Demonstrated:

1. **Asset-Class Detection**
   - Automatic detection from pair names
   - Asset-specific indicator windows
   - FX: SMA[10,20,50], RSI(14)
   - Crypto: SMA[5,10,20], RSI(7)

2. **Multi-GPU Support**
   - Automatic GPU detection
   - DataParallel wrapping
   - Memory monitoring

3. **Training Pipeline**
   - Real-time visualization
   - GPU utilization tracking
   - Progress monitoring

### Production Usage:

```python
# Prepare data with asset detection
python data/prepare_dataset.py --pairs btcusd,ethusd --t-in 120 --t-out 10

# Train with multi-GPU and AMP
python train/run_training.py \
  --pairs btcusd \
  --epochs 50 \
  --batch-size 256 \
  --use-amp \
  --device cuda

# Train A3C on GPU
python rl/run_a3c_training.py \
  --pair btcusd \
  --env-mode backtesting \
  --num-workers 8 \
  --device cuda
```

In [ ]:
# Final summary
print("\n" + "="*60)
print("SEQUENCE TRAINING PIPELINE - DEMONSTRATION COMPLETE")
print("="*60)
print("\n✅ Successfully demonstrated:")
print("   • Asset-class aware preprocessing")
print("   • Multi-GPU setup and monitoring")
print("   • Training with real-time visualization")
print("   • FX vs Crypto feature comparison")
print("\n🚀 Ready for production training!")
print("\n📚 Next steps:")
print("   1. Prepare your dataset with actual market data")
print("   2. Configure training parameters in TrainingConfig")
print("   3. Run training with multi-GPU and AMP enabled")
print("   4. Monitor performance and adjust hyperparameters")
print("="*60)